### Import Libraries

In [1]:
import numpy as np
from pathlib import Path
import tensorflow as tf
print(tf.__version__)

2023-05-27 14:34:40.504722: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-27 14:34:40.977642: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.12.0


### Set up data

In [13]:
data_name = 'data21-25_AC.npz'
data_path = Path('data/sat1') / data_name

with np.load(data_path) as f_data:
    data = f_data['data']
    labels = f_data['labels']
    participants = f_data['participants']
data = data.reshape(-1, 210, 30)

In [14]:
x_train = data[np.isin(participants, ['0021', '0022', '0023', '0024'])]
y_train = labels[np.isin(participants, ['0021', '0022', '0023', '0024'])]

x_test = data[np.isin(participants, ['0025'])]
y_test = data[np.isin(participants, ['0025'])]

In [15]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(210, 30)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_accuracy'])

In [20]:
model.fit(train_dataset, epochs=10)

Epoch 1/10


ValueError: in user code:

    File "/home/rick/miniconda3/envs/thesis/lib/python3.11/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/rick/miniconda3/envs/thesis/lib/python3.11/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/rick/miniconda3/envs/thesis/lib/python3.11/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/rick/miniconda3/envs/thesis/lib/python3.11/site-packages/keras/engine/training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "/home/rick/miniconda3/envs/thesis/lib/python3.11/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/rick/miniconda3/envs/thesis/lib/python3.11/site-packages/